In [1]:
import pandas as pd
import numpy as np
import datetime as dt

In [2]:
# load data
df = pd.read_csv('Adops & Data Scientist Sample Data - Q1 Analytics.csv')

In [3]:
df.head()

,ts,user_id,country_id,site_id
0,2019-02-01 00:01:24,LC36FC,TL6,N0OTG
1,2019-02-01 00:10:19,LC39B6,TL6,N0OTG
2,2019-02-01 00:21:50,LC3500,TL6,N0OTG
3,2019-02-01 00:22:50,LC374F,TL6,N0OTG
4,2019-02-01 00:23:44,LCC1C3,TL6,QGO3G


### Question 1.1

In [7]:
df_bdv = df[df.country_id == 'BDV'] # filter by country id equal BDV

#group by site id and count unique user_id
df_bdv_group = df_bdv.groupby('site_id')['user_id'].nunique().to_frame() 
# sort by nuique user number and then take the top one
df_bdv_group.sort_values('user_id',ascending = False).head(1)

,user_id
site_id,
5NPAU,544


### Question 1.2

In [8]:
df.ts = pd.to_datetime(df.ts) # convert ts to timestamp
begin_t = dt.datetime(2019,2,3,0,0,0) # 2019-02-03 00:00:00
end_t = dt.datetime(2019,2,4,23,59,59) # 2019-02-04 23:59:59

df_time = df.loc[(df.ts>begin_t)&(df.ts < end_t)] # filter by time
# for each user and site count visited times
df_count = df_time.groupby(['user_id','site_id'])['ts'].count().to_frame().reset_index()
df_count[df_count.ts > 10] # filter by visit times greater than 10

,user_id,site_id,ts
3,LC06C3,N0OTG,25
417,LC3A59,N0OTG,26
485,LC3C7E,3POLC,15
493,LC3C9D,N0OTG,17


### Question 1.3

In [9]:
# for each user, get last visit time
df_last_visit = df.groupby('user_id')['ts'].max().to_frame().reset_index()
# left join previous table with the full table to get last visit site
df_last_site = df_last_visit.merge(df, on = ['ts','user_id'], how='left' )
# change column names
df_last_site.columns = ['user_id','ts_last', 'country_last', 'site_last']
# group by last vist site and count each sites' occurrence, sort by counts
df_last_site.groupby('site_last')['ts_last'].count().to_frame().sort_values('ts_last', ascending = False)

,ts_last
site_last,
5NPAU,992
N0OTG,561
QGO3G,289
GVOFK,42
3POLC,28
RT9Z6,2
EUZ/Q,1
JSUUP,1


### Question 1.4

In [13]:
# for each user, get first visit time
df_first_visit = df.groupby('user_id')['ts'].min().to_frame().reset_index()
# left join previous table with the full table to get first visit site
df_first_site = df_first_visit.merge(df, on = ['ts','user_id'], how='left' )
# change column names
df_first_site.columns = ['user_id','ts_first', 'country_first', 'site_first']
# combine with the last visit table we get from question 1.3, join by using user_id
df_comb = df_last_site.merge(df_first_site, on ='user_id', how = 'left')
# filter by last visit site equal first visit site and compute the length
len(df_comb[df_comb.site_first == df_comb.site_last])

1670